Keys added:

* 'CID (canonical)': unique indetifier. Submission w/ same structure had same CID. Duplicates were removed —description combined.
* 'old_index': the order in which they were added in the postera site is kind of sequential
* 'clean_creator': the cleaned up creator field
* 'SMILES': this may be pre-reaction covalent
* 'new_smiles': smiles found in crystal
* 'fragments': inspirations. DIRTY AS OF 15/12/22. Not combined w/ cleaned set
* 'xcode': according to Fragalysis
* 'Structure ID': according to Postera
* 'site_name': fragalysis site
* 'pdb_entry'
* 'submission_date': from IC50 file
* 'inferred_submission_date': isotonic regression of the above
* 'order_date': true
* 'shipment_date': true 
* 'description': rationale + submission notes
* 'N_creator_submission': compounds in submission group N_creator_submission
* 'N_submission_group':
* 'resubmitted': was this submitted multiple times
* 'Enamine - REAL Space', 'Enamine - Extended REAL Space', 'Enamine - SCR', 'Enamine - BB', 'Mcule', 'Mcule Ultimate',
* 'N_chars', 'N_words', 'N_words_cutoff'
* keyword classification: 'classified_method'
* 'flesch', 'dale_chall': readability indices (>20 words or more)
* 'okay': keep?


## Retrieve data

In [1]:
!pip install -q PyGithub

In [2]:
import requests
import pandas as pd
from io import StringIO
from typing import IO
from github import Github

user = 'postera-ai'
repo_name = 'COVID_moonshot_submissions'
filename = 'downloaded_COVID_submissions_file.csv'

repo = Github().get_user(user).get_repo(repo_name)
spreadsheet_url = repo.get_contents(filename).download_url

# ----------------------------------------------------------------------------------------------------

response: requests.Response = requests.get(spreadsheet_url)
response.raise_for_status()
csv_handle: IO = StringIO(response.text)

# ----------------------------------------------------------------------------------------------------



moonshot = pd.read_csv(csv_handle, low_memory=False)
# the table contains multiple rows/submitted compounds per submission.
# Say the CID `ANT-DIA-3c79be55-1` finishes in `-1`
moonshot['CID_group']=moonshot['CID (canonical)'].fillna('').str.extract(r'(.*)\-\d+')

# force-standarise the dataframe!
_choices = {row.name: set(map(type, row.unique())) for i, row in moonshot.transpose().iterrows()}
moonshot = moonshot.assign(
    **{k: moonshot[k].fillna(False).astype(bool) for k, v in _choices.items() if bool in v},
    **{k: moonshot[k].fillna('').astype(str) for k, v in _choices.items() if str in v})

# defrag
moonshot = moonshot.copy()

# fix dates
for k in ('ORDER_DATE','SHIPMENT_DATE'):
    moonshot[k.lower()] = pd.to_datetime(moonshot[k], format='%Y-%m-%d', errors='coerce')

moonshot['order_month'] = moonshot['order_date'].apply(lambda d: d.month + 12 * (d.year -2020))
moonshot['shipment_month'] = moonshot['shipment_date'].apply(lambda d: d.month + 12 * (d.year -2020))

## Combine rationale and Submission notes
Reason: nobody understood the difference.

In [3]:
moonshot['description'] = (moonshot['rationale'].fillna('') +'. ' + moonshot['Submission Notes'].fillna('') +'. ')\
                    .astype(str)\
                    .str.strip()\
                    .str.replace('..', '', regex=False)

## Fix names
Folk were not overly consistent with their names...

In [4]:
import re

class NameFixer:
    lab_heads = {'London Lab': 'Nir London', 
                 'Chodera Lab': 'John Chodera',
                'Lee Lab': 'Alpha Lee',
                'Trippier': '?? Trippier', 
                'Volkamer': '?? Volkamer'}
    known = {'Warren': 'Warren Thompson',
             'Jag': 'Jag Heer',
             'Jan': 'Jan Hullaert'}
    weirds = ['Med-Chem team', 'INSCoV',]
    redirect = {'Med Chemists Group': 'Med-Chem team',
               'Tatiana': 'Tetiana Matviyuk'}
    
    def __init__(self):
        self.unmatched = set()
        self.single_names = set()
        
    def __call__(self, name: str) -> str:
        """
        Find Matt Robinson. Remove titles. Remove initials.
        Skip single name folk
        """
        # weird id set
        for weird in self.weirds:
            if weird in name:
                return weird
            
        s = name.title()
        if s in self.redirect:
            return self.redirect[s]
        for r in self.redirect:
            if r in s:
                return self.redirect[r]
        
        if s in ('-', '', ' ', 'Anon', 'Unk'):
            return 'anonymous'
        
        # clean
        s = s.replace('Matt ', 'Matthew ')
        s = re.sub(r'^Dr.?', '', s)
        s = re.sub(r'^Prof.?', '', s)
        s = re.sub(r'^(\w)\.? ', r'\1', s)
        s = re.sub(r'(\w)-(\w)', r'\1\2', s)
        s = re.sub(r' (\w)\.? ', r' ', s)
        s = re.sub(r' (\w) ', r' ', s)
        s = re.sub(r'\s+', r' ', s)
        s = re.sub(r' And .*', r'', s)  # title makes and -> And
        s = re.sub(r'Inscov.*', 'Inscov', s).strip()
        
        r = re.match(r'^([ \w]+)', s)
        
        if not r:
            self.unmatched.add(name)
            return ''
        
        s = r.group(1).strip()
        if 'Lab' in s and s in self.lab_heads:
            return self.lab_heads[s]
        if s in self.redirect:
            return self.redirect[s]
        if s.count(' ') != 0:
            return s
        if s in self.known:
            return self.known[s]
        self.single_names.add(s)
        r = re.match(r'^([ \w]+)[,.&] ([ \w]+)', name)
        if not r:
            self.unmatched.add(name)
            return 'anonymous'
        
        return r.group(1).strip()+' '+r.group(2).strip()
    
    def revisit(self, name, series):
        return series.loc[series.str.contains(name)].drop_duplicates()

# debug:
# dejunk = NameFixer()
# for name in ('Ed J Griffen, MedChemica Ltd', 'Jan, Ghent University', 'Ed, Griffen', 'K. Takahashi'):
#     print( name, '>', dejunk(name) )

dejunk = NameFixer()
moonshot['clean_creator'] = moonshot.creator.fillna('').astype(str).apply(dejunk)
sum(moonshot['clean_creator'] == ''), len(moonshot)

(0, 20997)

In [5]:
moonshot['N_creator_submission'] = moonshot['clean_creator'].map(moonshot['clean_creator'].value_counts())
moonshot['N_submission_group'] = moonshot['CID_group'].map(moonshot['CID_group'].value_counts())

## remove duplicate CIDs

This is a resubmission by wheel reinvention of error in submitter form filling

In [6]:
import numpy as np
from collections import defaultdict

moonshot['resubmitted'] = moonshot.duplicated(subset='CID (canonical)', keep=False)


words = defaultdict(str)
fragments = defaultdict(set)
shipment = defaultdict(list)
ordered = defaultdict(list)
assayed = defaultdict(bool)
made = defaultdict(bool)
for i, row in moonshot.loc[moonshot['resubmitted']].iterrows():
    cid = row['CID (canonical)']
    words[cid] += '. ' + row.description
    if row.fragments.strip() != 'x0072':
        fragments[cid].update(row.fragments.split(','))
    assayed[cid] = assayed[cid] or row.ASSAYED
    made[cid] = assayed[cid] or row.MADE
    
# #
moonshot['old_index'] = moonshot.index
moonshot = moonshot.drop_duplicates('CID (canonical)').set_index('CID (canonical)', drop=False)
for cid in words:
    moonshot.at[cid, 'description'] = ','.join(words[cid])
    moonshot.at[cid, 'fragments'] = ','.join(fragments[cid])
    moonshot.at[cid, 'MADE'] = made[cid]
    moonshot.at[cid, 'ASSAYED'] = assayed[cid]

## Include current Fragalysis status

This is because the followup details were not updated.

In [7]:
import io
import requests

response = requests.get('https://fragalysis.diamond.ac.uk/api/targets/?title=Mpro')
meta_url = response.json()['results'][0]['metadata']
response = meta_url = response.json()['results'][0]['metadata']
response = requests.get(meta_url)
metadata = pd.read_csv(io.StringIO(response.text), index_col=0)
in_fragalysis = metadata.alternate_name.to_list()

moonshot['in_fragalysis'] = moonshot['CID (canonical)'].isin(in_fragalysis)
metadata['xcode'] = metadata['RealCrystalName'].str.split('-', expand=True)[1]
for k in ('site_name', 'pdb_entry', 'new_smiles', 'xcode'):
    moonshot[k] = moonshot['CID (canonical)'].map(metadata.set_index('alternate_name')[k].to_dict()).fillna('')

# px.scatter(moonshot, 'shipment_date', 'submission_date', color='in_fragalysis', opacity=0.2, 
#            title='Relationship between submission and shipment dates')

## Include submission date
Additionally add latest pIC50
The file used for the cross referencing is not shared herein due to permissions.

In [29]:
import pandas as pd
from datetime import datetime

dated = pd.read_csv('moonshot_dated_data.csv', index_col=0)
# https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
for k in ('SUBMITTED_DATE','ORDERED_DATE'):
    dated[k.lower()] = pd.to_datetime(dated[k], format='%Y%m%d')
import re

def fix_ic50(value):
    if str(value) == 'nan':
        return 100
    if isinstance(value, float):
        return value
    rex = re.search('([\d.]+)', str(value))
    if rex:
        return float(rex.group(1))
    return 100

dated['IC50'] = dated['ProteaseAssay_Fluorescence_Dose-Response_Weizmann: Avg IC50 (uM)'].apply(fix_ic50)
moonshot['IC50'] = moonshot['CID (canonical)'].map(dated.IC50)
#moonshot['pIC50'] = moonshot['IC50'].apply(np.log10)
# IC50 unit is µM hence the +6
moonshot['pIC50'] = moonshot.IC50.apply(lambda i: -np.log10(i) + 6)

moonshot['submission_date'] = moonshot['CID (canonical)'].map(dated.submitted_date)
#moonshot['shipment_date'] = pd.to_datetime(moonshot.SHIPMENT_DATE, format='%Y-%m-%d')

### Fill empties
Now things get hairy: missing values in submission dates.
This is because the data comes from the IC50 table (1244 vs. 20,997).
Two options:

* value from previous using `.fillna(method='ffill')`
* monotonic regression

The former may cause issues.
Monotonic regression is > 1 month over for 6.7% as some clear outliers exist.

In [9]:
# this is poor choice:
#moonshot['likely_submission_date'] = moonshot['submission_date'].fillna(method='bfill').fillna(method='ffill')
#moonshot['likely_submission_month'] = moonshot['likely_submission_date'].apply(lambda d: d.month + 12 * (d.year -2020))

from sklearn.isotonic import IsotonicRegression
from datetime import date, timedelta

iso = IsotonicRegression(out_of_bounds='clip')
iso.fit(*list(zip(*[(i,v.toordinal()) for i,v in enumerate(moonshot.submission_date) if str(v) != 'NaT'])))
moonshot['inferred_submission_date'] = pd.Series(iso.predict(np.arange(len(moonshot.submission_date)))).astype(int).apply(datetime.fromordinal).values
moonshot['inferred_submission_month'] = moonshot['inferred_submission_date'].apply(lambda d: d.month + 12 * (d.year -2020))

print('Fraction under', sum(moonshot.submission_date + timedelta(days=30) < (moonshot.inferred_submission_date))/sum(~moonshot.submission_date.isna()))
print('Fraction over', sum(moonshot.submission_date >= moonshot.inferred_submission_date + timedelta(days=1))/sum(~moonshot.submission_date.isna()))

Fraction under 0.0
Fraction over 0.0


##  Simple word counts etc.

In [10]:
import numpy as np

moonshot['N_chars'] = moonshot.description.apply(len)
moonshot['N_words'] = moonshot.description.apply(lambda x: len(re.findall(r'[\w-]+',x)))

# Some submitters copypasted abstracts...
moonshot['N_words_cutoff'] = moonshot.N_words.apply(lambda v: v if v < 500 else float('nan'))


# use `in_fragalysis`
# moonshot['CRYSTALLISED'] = moonshot['Structure ID'] != ''
# moonshot['crystallised'] = moonshot['Structure ID'] != ''

## Term classification
This is the result of a manual inspection of way too many entries

In [25]:
import enum, operator

class Method(enum.Enum):
    INITIAL = enum.auto()  # these were possibly poised libary — not checked
    OLD = enum.auto()  # original SARS inhibitor
    MANUAL = enum.auto()
    MANUAL_POSSIBLY = enum.auto()
    DOCKING = enum.auto()
    FEP = enum.auto()
    UNKNOWN = enum.auto()


def classify(row):
    def has_any_term(*terms):
        return any([term in row.description.lower() for term in terms])
        
    if has_any_term('missing fragalysis structures',
                    'first batch of fragments so we have a moonshot cid for them',
                    'fragment that was missing an id not a design but a frament',
                   'second batch of submissions of fragments in order to generate moonshot cid',
                   'final set of fragments so we can generate moonshot cid'):
        return Method.INITIAL
    if has_any_term('sars inhibitor'):
        return Method.OLD
    if has_any_term('fep'):
        return Method.FEP
    if has_any_term('dock', 'seesar', 'vina', 'autodock', 'screen', 'drug-hunter', 'search'):
        return Method.DOCKING
    if has_any_term('by-eye', 'merg', 'link', 'coupl'):
        return Method.MANUAL
    if has_any_term('swap', 'racem', 'side product', 'intermediate', 'break',
                    'bioisoster', 'isomer', 'around', 'replace', 'isomer', 'enantiomer',
                    'introduction', 'substitution', 'shifted', 'combo',
                    'expansion', 'made by', 'design', 'idea', 'based', 'pairs',
                    'modification', 'derivative', 'common sense', 'suggested',
                    'similar to', 'analogues', 'easy to make', 'exploration', 'inspir', 'possible'):
        return Method.MANUAL_POSSIBLY
    if len(row.description) > 200:
        return Method.MANUAL_POSSIBLY 
    if row.fragments != 'x0072':
        return Method.MANUAL_POSSIBLY
    return Method.UNKNOWN
    
moonshot['classified_method'] = moonshot.apply(classify, axis=1).apply(operator.attrgetter('name'))

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/matteo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
penta_words = (moonshot.description + '. ') * 5

from readability import Readability
from readability.exceptions import ReadabilityException

def flesch_score(text: str) -> float:
    try:
        return Readability(text).flesch_kincaid().score
    except ReadabilityException as error:
        return np.nan
    
def dale_score(text: str) -> float:
    try:
        return Readability(text).dale_chall().score
    except ReadabilityException as error:
        return np.nan
    
moonshot['flesch'] = penta_words.apply(flesch_score)
moonshot['dale_chall'] = penta_words.apply(dale_score)

In [26]:
moonshot['initial_screen'] = moonshot.site_name.str.contains('XChem Screen')

for i in moonshot.index[moonshot.initial_screen]:
    moonshot.at[i, 'classified_method'] = 'STARTING_LIBRARY'
    moonshot.at[i, 'clean_creator'] = 'DSi-Poised Library'
    for k in ('submission_date','inferred_submission_date','shipment_date'):
        moonshot.at[i, k] = pd.Timestamp(year=2020, month=2, day=1)
    
for  i in moonshot.index[moonshot.description.str.contains('SARS inhibitors')]:
    moonshot.at[i, 'classified_method'] = 'PRIOR_SARS_INHIBITOR'

moonshot['okay'] = ~moonshot.clean_creator.isin(['Mark Davies', 'Maksym Voznyy'])

In [30]:
moonshot.copy().to_pickle('moonshot_submissions.p')
moonshot.to_csv('moonshot_submissions.csv')

In [31]:
moonshot[['CID (canonical)','CID_group', 'old_index', 'clean_creator', 'SMILES', 'new_smiles',
                           'fragments', 'xcode', 'Structure ID', 'xcode','site_name', 'pdb_entry',
                            'ORDERED', 'MADE', 'ASSAYED', 'in_fragalysis',
                            'IC50', 'pIC50',
                           'submission_date', 'inferred_submission_date', 'order_date', 'shipment_date', 
                           'description',
                           'N_creator_submission', 'N_submission_group', 'resubmitted',
                           'Enamine - REAL Space', 'Enamine - Extended REAL Space',
                           'Enamine - SCR', 'Enamine - BB', 'Mcule', 'Mcule Ultimate',
                           'N_chars', 'N_words', 'N_words_cutoff', 'classified_method', 'flesch',
                           'dale_chall', 'okay']].to_csv('moonshot_submissions.min.csv', index=False)